# AC Optimial Power FLow (ACOPF) - Julia JuMP
This repository contains the supplementary material for the paper entitled "On the comparison of different convexified power flow models in radial network" published on IEEE Xplore: https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9814764

Implementation on Python - CVXPY can be found at: https://github.com/HieuNguyenNCAT/Distribution-Power-Flows-Using-Connic-Programming

In the repsitory, the code of following sections are provided:
1. Read data from MATPOWER
2. Bus injection model
3. Branch flows model

# Libaries Declaration

In [2]:
using JuMP
using LinearAlgebra
using Random

# using CUDA
# using CUDA_jll
# using SCS
using CPLEX
using Gurobi
using MATLAB
using MAT

# Read data from MATPOWER

In [3]:
filedata = matopen("Radial_Netdata.mat")
data = matread("Radial_Netdata.mat")
#Default = case33bw
case = "case33bw"

r = data["r"]
x = data["x"]
slack_bus = 1 #Julia indexing starts from 1, not as Python to starts from 1
I = length(data["Pd"])
G = data["G"]
B = -data["B"]
MVAlim = data["MVAlim"]
status = data["status"]


# println(L)

32×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

Define a connex function to check whether a pair of node $i,j=\ell \in \mathcal{L}$

In [4]:
# Define connex function
function connex(i,j,data) #return 1 of (i,j) or (j,i) is a "connected" line
    temp = false
    for line in 1:32
        z1 = (i == data["from_bus"][line]) && (j == data["to_bus"][line]) #Julia indexing starts from 1
        z2 = (j == data["from_bus"][line]) && (i == data["to_bus"][line])
        z3 = (data["status"][line]==1) #some lines are open switches
        z  = (z1 || z2) && z3
        temp = temp || z
    end
    return temp
end

connex (generic function with 1 method)

Normalize the data from MATPOWER

In [5]:
print(MVAlim)
if case == "case18"
    scale = 10
else
    scale = 1
end
data["Sbase"] = scale*data["Sbase"]
r = r*scale
x = x*scale
data["gs"] = data["gs"]/scale
data["bs"] = data["bs"]/scale

println(r)

[270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0; 270.0;;][0.05752591161723931; 0.3075951673242839; 0.22835665566062455; 0.23777792751984705; 0.5109948114372992; 0.11679881404281126; 0.4438604503742304; 0.642643047350938; 0.6513780013926013; 0.12266371175649943; 0.2335976280856225; 0.9159223237972591; 0.33791793635462913; 0.36873984561592654; 0.4656354429495194; 0.8042396971217077; 0.4567133113212491; 0.10232374734519789; 0.9385084192478456; 0.2554974057186496; 0.4423006371525048; 0.28151509025703225; 0.5602849092438275; 0.559037058666447; 0.12665683360411692; 0.17731956704576368; 0.6607368807229547; 0.5017607171646838; 0.3166420840102922; 0.6079528012997611; 0.19372880213831675; 0.2127585234433688;;]


# Modelling

## Model and variables definition

Let's define variables: $P_{ij}, Q_{ij}, R_\ell, T_\ell, V^{sq}_i, I^{sq}_\ell$,
and define the constraint set

In [6]:
model = Model(optimizer_with_attributes(Gurobi.Optimizer))

@variable(model, P[i=1:I,j=1:I]);
@variable(model, Q[i=1:I,j=1:I]);
@variable(model, R[1:32] >= 0);
@variable(model, T[1:32]);
@variable(model, Vsq[i=1:I]);
@variable(model, Isq[i=1:I] >= 0);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-05


## Voltage limits constraint:
$(\underline{V}_i)^2 \leq V_{i}^{sq} \leq (\overline{V}_i)^2 , \quad \forall i \in \mathcal{I}$

In [7]:
for i in 1:I
    if i != slack_bus
       @constraint(model, (data["Vmin"][i])^2 <= Vsq[i] <= (data["Vmax"][i])^2)
    end
end

if case == "case18"
    @constraint(model, Vsq[slack_bus] == 1.1)
else
    @constraint(model, Vsq[slack_bus] == 1)
end

Vsq[1] == 1.0

## The line power flow equations $P_{ij}, P_{ji}, Q_{ij}, Q_{ji}$ and the squared line current $I^{sq}_\ell$:  
$P_{ij} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( r_{\ell} V_{i}^{sq} - r_\ell R_{\ell} + x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$Q_{ij} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( x_{\ell} V_{i}^{sq} - x_\ell R_{\ell} - x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$P_{ji} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( r_{\ell} V_{j}^{sq} - r_\ell R_{\ell} - x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$Q_{ji} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( x_{\ell} V_{j}^{sq} - x_\ell R_{\ell} + x_\ell T_{\ell} \right), \quad \forall \ell = ij \in \mathcal{L}$

$I^{sq}_{\ell} = \frac{1}{r^2_{\ell} + x^2_\ell} \left( V_{i}^{sq} + V_{j}^{sq} - 2R_{\ell} \right), \quad  \forall \ell = ij \in \mathcal{L}$

In [8]:
for line in 1:32
    i = floor(Int,data["from_bus"][line])
    j = floor(Int,data["to_bus"][line])
    
#     println(i)
#     println(j)
    if status[line] == 1
        @constraint(model, P[i,j] == (r[line]*Vsq[i] - r[line]*R[line]+ x[line]*T[line])/(r[line]^2 + x[line]^2))
        @constraint(model, Q[i,j] == (x[line]*Vsq[i] - x[line]*R[line]- r[line]*T[line])/(r[line]^2 + x[line]^2))
        @constraint(model, P[j,i] == (r[line]*Vsq[j] - r[line]*R[line]- x[line]*T[line])/(r[line]^2 + x[line]^2))
        @constraint(model, Q[j,i] == (x[line]*Vsq[j] - x[line]*R[line]+ r[line]*T[line])/(r[line]^2 + x[line]^2))
        @constraint(model, Isq[line]  == (Vsq[i]+Vsq[j] - 2*R[line])/(r[line]^2 + x[line]^2))
    end
end

## The active and reactive power balance constraints:  
$P^d_{i} + \sum_{j \in \mathcal{N} \left(i \right)} P_{ij} = 0, \quad \forall i \in \mathcal{I}$

$Q^d_{i} + \sum_{j \in \mathcal{N} \left( i \right)} Q_{ij} = 0, \quad \forall i \in \mathcal{I}$

In [9]:
bus_Pinjection = [-sum(P[i,j] for j in 1:I if connex(i,j,data)) for i in 1:I]
bus_Qinjection = [-sum(Q[i,j] for j in 1:I if connex(i,j,data)) for i in 1:I]

for i in 1:I
    if i != slack_bus
        @constraint(model, bus_Pinjection[i] + data["gs"][i]*Vsq[i] == data["Pd"][i]/data["Sbase"])
        @constraint(model, bus_Qinjection[i] + data["bs"][i]*Vsq[i] == data["Qd"][i]/data["Sbase"])
    end
end

## The SOC constraint:  
The rotated conic constraint $(R_{\ell})^2 + (T_{\ell})^2 \leq V_{i}^{sq} V_{j}^{sq}, ~ R_{\ell}  \geq 0,  \quad \forall \ell = ij \in \mathcal{L}$ 
can be implemented by 



In [10]:
for line in 1:32
    if status[line] == 1
        i = floor(Int,data["from_bus"][line])
        j = floor(Int,data["to_bus"][line])
        
        # Rotated cone syntax: [t,x,u1,u2] in RotatedSecondOrderCone() means norm2(u1,u2) <= 2tx
        @constraint(model, [Vsq[i], 0.5*Vsq[j], R[line], T[line]] in RotatedSecondOrderCone())
        
        # Second-order cone syntax: [y,x1,x2,x3] in SecondOrderCone() means norm2(x1,x2,x3) <= y
#         @constraint(model, [0.5*(Vsq[i] + Vsq[j]), R[line], T[line], 0.5*(Vsq[i] - Vsq[j])] in SecondOrderCone())
        
        # Quadratic constraint
#         @constraint(model, R[line]^2 + T[line]^2 + (0.5*(Vsq[i] - Vsq[j]))^2 <= (0.5*(Vsq[i] + Vsq[j]))^2)
    end
end

## Objective function: $\min P^{\sf grid}$

In this paper, we tested the model with CPLEX and MOSEK. Many other solvers can be called by JuMP if installed.

In [11]:
Pgrid = -bus_Pinjection[slack_bus]*data["Sbase"]
Qgrid = -bus_Qinjection[slack_bus]*data["Sbase"]

@objective(model, Min, Pgrid)
# Solve statement
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 385 rows, 2468 columns and 1151 nonzeros
Model fingerprint: 0x7f334be6
Model has 32 quadratic constraints
Coefficient statistics:
  Matrix range     [4e-01, 5e+02]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+02, 1e+02]
  Bounds range     [8e-01, 1e+00]
  RHS range        [1e-04, 1e+00]
Presolve removed 233 rows and 2316 columns
Presolve time: 0.02s
Presolved: 192 rows, 156 columns, 641 nonzeros
Presolved model has 32 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.236e+03
 Factor NZ  : 2.072e+03
 Factor Ops : 2.537e+04 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.25820284e+04 -8.04603665e+03  2.16e+00 9.59e+02  6.93e+02     0s
   1   1.826773

## Results

In [12]:
@show objective_value(model);
@show value.(P);

objective_value(model) = 3.9177161998591146
value.(P) = [0.0 0.039177161998614096 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; -0.039054759797599314 0.0 0.03444331202116002 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0036114477764392916 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.03392538486573425 0.0 0.02362924569465763 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.009396139171076623 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.023430233984585457 0.0 0.022230233984585457 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.022043236452943104 0.0 0.021443236452943104 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.021060735615109082 0.0 0.010952849401251963 0.0 0.0 0.0 0.0 0.0 0.0 0.